In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

In [4]:
df=df.drop(columns=['Unnamed: 32','id'])

In [5]:
df.shape

(569, 31)

In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
X=df.drop(columns=['diagnosis'])
y=df['diagnosis']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.transform(y_test)

In [9]:
X_test

array([[-0.46649743, -0.13728933, -0.44421138, ..., -0.19435087,
         0.17275669,  0.20372995],
       [ 1.36536344,  0.49866473,  1.30551088, ...,  0.99177862,
        -0.561211  , -1.00838949],
       [ 0.38006578,  0.06921974,  0.40410139, ...,  0.57035018,
        -0.10783139, -0.20629287],
       ...,
       [-0.73547237, -0.99852603, -0.74138839, ..., -0.27741059,
        -0.3820785 , -0.32408328],
       [ 0.02898271,  2.0334026 ,  0.0274851 , ..., -0.49027026,
        -1.60905688, -0.33137507],
       [ 1.87216885,  2.80077153,  1.80354992, ...,  0.7925579 ,
        -0.05868885, -0.09467243]])

In [10]:
import torch
X_train_tensor=torch.from_numpy(X_train)
X_test_tensor=torch.from_numpy(X_test)
y_train_tensor=torch.from_numpy(y_train)
y_test_tensor=torch.from_numpy(y_test)

In [11]:
X_train_tensor

tensor([[-1.4408, -0.4353, -1.3621,  ...,  0.9320,  2.0972,  1.8865],
        [ 1.9741,  1.7330,  2.0917,  ...,  2.6989,  1.8912,  2.4978],
        [-1.4000, -1.2496, -1.3452,  ..., -0.9702,  0.5976,  0.0579],
        ...,
        [ 0.0488, -0.5550, -0.0651,  ..., -1.2390, -0.7086, -1.2715],
        [-0.0390,  0.1021, -0.0314,  ...,  1.0500,  0.4343,  1.2134],
        [-0.5486,  0.3133, -0.6035,  ..., -0.6110, -0.3345, -0.8463]],
       dtype=torch.float64)

## defining the model

In [34]:
class MyNeuralNetwork():
  def __init__(self,X):
    self.weights=torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
    self.bias=torch.rand(1, dtype=torch.float64,requires_grad=True)

  def forward(self,X):
    z=torch.matmul(X,self.weights)+self.bias
    y_pred=torch.sigmoid(z)
    return y_pred

  def loss_function(self,y_pred,y):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    # Calculate loss binary_cross_entropy
    train_loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
    return train_loss


In [35]:
learning_rate=0.3
epochs=25


In [36]:
model=MyNeuralNetwork(X_train_tensor)
for epoch in range(epochs):
  y_pred=model.forward(X_train_tensor)
  train_loss=model.loss_function(y_pred,y_train_tensor)

  train_loss.backward()

  with torch.no_grad():
    model.weights-=learning_rate* model.weights.grad
    model.bias-=learning_rate*model.bias.grad

    model.weights.grad.zero_()
    model.bias.grad.zero_()

    print(f'Epoch: {epoch + 1}, Loss: {train_loss.item()}')


Epoch: 1, Loss: 3.79782848275663
Epoch: 2, Loss: 3.4319647206339736
Epoch: 3, Loss: 3.056939868602774
Epoch: 4, Loss: 2.6532796800335388
Epoch: 5, Loss: 2.2079148478382433
Epoch: 6, Loss: 1.7732268135208065
Epoch: 7, Loss: 1.4004200220625034
Epoch: 8, Loss: 1.1439301619491096
Epoch: 9, Loss: 1.0037686401408528
Epoch: 10, Loss: 0.9375487723840832
Epoch: 11, Loss: 0.897309334754285
Epoch: 12, Loss: 0.8658670935208297
Epoch: 13, Loss: 0.8391711439589894
Epoch: 14, Loss: 0.8162027682144084
Epoch: 15, Loss: 0.796465035914859
Epoch: 16, Loss: 0.77955412381093
Epoch: 17, Loss: 0.7650981215572188
Epoch: 18, Loss: 0.7527526491104455
Epoch: 19, Loss: 0.7422056806207064
Epoch: 20, Loss: 0.73318181104417
Epoch: 21, Loss: 0.7254432767424885
Epoch: 22, Loss: 0.7187879188038989
Epoch: 23, Loss: 0.7130453810222015
Epoch: 24, Loss: 0.7080727773820596
Epoch: 25, Loss: 0.7037505646758391


In [37]:
## evaluating the model
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.6).float()

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_tensor,y_pred)

0.6754385964912281